In [2]:
%load_ext line_profiler
import line_profiler

In [4]:
from Cython.Compiler.Options import get_directive_defaults
directive_defaults = get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [24]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [93]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
from re import M
import numpy as np
cimport numpy as np
from numpy cimport ndarray 
cimport cython
np.import_array()
from scipy.sparse import csc_matrix

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
cdef get_A(double j,double m,str type=""):
    if type == "+":
        return np.sqrt((j-m)*(j+m+1))
    elif type == "-":
        return np.sqrt((j+m)*(j-m+1))
    else:
        return m

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
cdef get_B(double j,double m,str type=""):
    if type == "+":
        return np.sqrt((j-m)*(j-m-1))
    elif type == "-":
        return -np.sqrt((j+m)*(j+m-1))
    else:
        return np.sqrt((j+m)*(j-m))

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
cdef get_D(double j,double m,str type=""):
    if type == "+":
        return -np.sqrt((j+m+1)*(j+m+2))
    if type == "-":
        return np.sqrt((j-m+1)*(j-m+2))
    else:
        return (j+m+1)*(j-m+1)

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
cdef get_Lambda(int N, double j,str type=""):
    if type == "a":
        return (N/2+1)/(2*j*(j+1))
    elif type == "b":
        return (N/2+j+1)/(2*j*(2*j+1))
    else:
        return (N/2-j)/(2*(j+1)*(2*j+1))

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
cdef dicke_bx(int N,dict jmm1,ik,dim):
    # create a dicke basis follow jmm1
    # jmm1 as {(j,m,m1):p}
    cdef int i,k 
    cdef np.ndarray[np.complex128_t, ndim=2] rho = np.zeros((dim,dim),dtype = np.complex128)
    for key in jmm1:
        i,k = ik[key]
        rho[i,k] = jmm1[key]
    return csc_matrix(rho)

@cython.exceptval(-2, check=True)
@cython.boundscheck(False)
def calc_rho_0(iks,jmm1,state,all_iks,j_min,j_max,N_in,d_dicke):

    accumulate_states = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
    cdef int i,k 
    cdef double j,m,m1,Lambda_a,Lambda_b,B_jm_z,B_jm1_z,D_jm_z,D_jm1_z,Lambda_d,A_jm_plus,A_jm1_minus,B_jm_minus,B_jm1_minus,D_jm_plus,D_jm1_plus
    cdef complex p_jmm1 
    
    for ik in iks:
        j,m,m1 = jmm1[ik]
        i,k = ik
        first_term = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        second_term = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        third_term = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_1 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128) 
        gamma_2 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_3 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128) 
        gamma_4 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_5 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_6 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_7 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        gamma_8 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128) 
        gamma_9 = csc_matrix((d_dicke, d_dicke), dtype=np.complex128)
        p_jmm1 = state[i,k]

        if j >= j_min and j <= j_max and m > -j and m1 > -j and m < j and m1 < j:
            Lambda_a = get_Lambda(N_in,j,type="a")
            gamma_1 = m*m1*Lambda_a*dicke_bx(N_in,{(j,m,m1):1},all_iks,d_dicke)

        if (j-1) >= j_min and (j-1) <= j_max and (m >= -(j-1) and m1 >= -(j-1)) and (m <= (j-1) and m1 <= (j-1)):
            B_jm_z = get_B(j,m,type="z")
            B_jm1_z = get_B(j,m1,type="z")
            Lambda_b = get_Lambda(N_in,j,type="b")
            gamma_2 = B_jm_z*B_jm1_z*Lambda_b*dicke_bx(N_in,{(j-1,m,m1):1},all_iks,d_dicke) 

        if (j+1) <= j_max and (j+1) >= j_min and (m >= -(j+1) and m1 >= -(j+1)) and (m <= (j+1) and m1 <= (j+1)):
            D_jm_z = get_D(j,m,type="z")
            D_jm1_z = get_D(j,m1,type="z")
            Lambda_d = get_Lambda(N_in,j,type="d")
            gamma_3 = D_jm_z*D_jm1_z*Lambda_d*dicke_bx(N_in,{(j+1,m,m1):1},all_iks,d_dicke)
        
        if j >= j_min and j <= j_max  and (m-1) >= -j and (m1-1) >= -j and (m-1) <= j and (m1-1) <= j:
            A_jm_minus = get_A(j,m,type="-")
            A_jm1_minus = get_A(j,m1,type="-")
            Lambda_a = get_Lambda(N_in,j,type="a")
            gamma_4 = A_jm_minus*A_jm1_minus*Lambda_a*dicke_bx(N_in,{(j,m-1,m1-1):1},all_iks,d_dicke)
        
        if (j-1) >= j_min and (j-1) <= j_max and (m-1) >= -(j-1) and (m1-1) >= -(j-1) and (m-1) <= (j-1) and  (m1-1) <= (j-1):
            B_jm_minus = get_B(j,m,type="-")
            B_jm1_minus = get_B(j,m1,type="-")
            Lambda_b = get_Lambda(N_in,j,type="b")
            gamma_5 = B_jm_minus*B_jm1_minus*Lambda_b*dicke_bx(N_in,{(j-1,m-1,m1-1):1},all_iks,d_dicke)
        
        if  (j+1) >= j_min and (j+1) <= j_max and (m-1) >= -(j+1) and (m1-1) >= -(j+1) and (m-1) <= (j+1) and (m1-1)  <= (j+1):
            D_jm_minus = get_D(j,m,type="-")
            D_jm1_minus = get_D(j,m1,type="-")
            Lambda_d = get_Lambda(N_in,j,type="d")
            gamma_6 = D_jm_minus*D_jm1_minus*Lambda_d*dicke_bx(N_in,{(j+1,m-1,m1-1):1},all_iks,d_dicke)
        
        if j >= j_min and j <= j_max and (m+1) <= j and (m1+1) <= j and (m+1) >= -j and (m1+1) >= -j:
            A_jm_plus = get_A(j,m,type="+")
            A_jm1_plus = get_A(j,m1,type="+")
            Lambda_a = get_Lambda(N_in,j,type="a")
            gamma_7 = A_jm_plus*A_jm1_plus*Lambda_a*dicke_bx(N_in,{(j,m+1,m1+1):1},all_iks,d_dicke)
        
        if (j-1) >= j_min and (j-1) <= j_max and (m+1) >= -(j-1) and (m1+1) >= -(j-1) and (m+1) <= (j-1) and (m1+1) <= (j-1):
            B_jm_plus = get_B(j,m,type="+")
            B_jm1_plus = get_B(j,m1,type="+")
            Lambda_b = get_Lambda(N_in,j,type="b")
            gamma_8 = B_jm_plus*B_jm1_plus*Lambda_b*dicke_bx(N_in,{(j-1,m+1,m1+1):1},all_iks,d_dicke)
        
        if (j+1) >= j_min and (j+1) <= j_max and  (m+1) >= -(j+1) and (m1+1) >= -(j+1) and (m+1) <= (j+1) and (m1+1) <= (j+1):
            D_jm_plus = get_D(j,m,type="+")
            D_jm1_plus = get_D(j,m1,type="+")
            Lambda_d = get_Lambda(N_in,j,type="d")
            gamma_9 = D_jm_plus*D_jm1_plus*Lambda_d*dicke_bx(N_in,{(j+1,m+1,m1+1):1},all_iks,d_dicke)

        accumulate_states += p_jmm1*((gamma_1+gamma_2+gamma_3)+(gamma_4+gamma_5+gamma_6)/2+(gamma_7+gamma_8+gamma_9)/2)

    return accumulate_states

In [94]:
import numpy as np 
from scipy.sparse import csc_matrix
iks = [(10, 10)]
jmm1 = {(35, 35): (0.0, -0.0, -0.0), (34, 34): (1.0, -1.0, -1.0), (34, 33): (1.0, -1.0, 0.0), (34, 32): (1.0, -1.0, 1.0), (33, 34): (1.0, 0.0, -1.0), (33, 33): (1.0, 0.0, 0.0), (33, 32): (1.0, 0.0, 1.0), (32, 34): (1.0, 1.0, -1.0), (32, 33): (1.0, 1.0, 0.0), (32, 32): (1.0, 1.0, 1.0), (31, 31): (2.0, -2.0, -2.0), (31, 30): (2.0, -2.0, -1.0), (31, 29): (2.0, -2.0, 0.0), (31, 28): (2.0, -2.0, 1.0), (31, 27): (2.0, -2.0, 2.0), (30, 31): (2.0, -1.0, -2.0), (30, 30): (2.0, -1.0, -1.0), (30, 29): (2.0, -1.0, 0.0), (30, 28): (2.0, -1.0, 1.0), (30, 27): (2.0, -1.0, 2.0), (29, 31): (2.0, 0.0, -2.0), (29, 30): (2.0, 0.0, -1.0), (29, 29): (2.0, 0.0, 0.0), (29, 28): (2.0, 0.0, 1.0), (29, 27): (2.0, 0.0, 2.0), (28, 31): (2.0, 1.0, -2.0), (28, 30): (2.0, 1.0, -1.0), (28, 29): (2.0, 1.0, 0.0), (28, 28): (2.0, 1.0, 1.0), (28, 27): (2.0, 1.0, 2.0), (27, 31): (2.0, 2.0, -2.0), (27, 30): (2.0, 2.0, -1.0), (27, 29): (2.0, 2.0, 0.0), (27, 28): (2.0, 2.0, 1.0), (27, 27): (2.0, 2.0, 2.0), (26, 26): (3.0, -3.0, -3.0), (26, 25): (3.0, -3.0, -2.0), (26, 24): (3.0, -3.0, -1.0), (26, 23): (3.0, -3.0, 0.0), (26, 22): (3.0, -3.0, 1.0), (26, 21): (3.0, -3.0, 2.0), (26, 20): (3.0, -3.0, 3.0), (25, 26): (3.0, -2.0, -3.0), (25, 25): (3.0, -2.0, -2.0), (25, 24): (3.0, -2.0, -1.0), (25, 23): (3.0, -2.0, 0.0), (25, 22): (3.0, -2.0, 1.0), (25, 21): (3.0, -2.0, 2.0), (25, 20): (3.0, -2.0, 3.0), (24, 26): (3.0, -1.0, -3.0), (24, 25): (3.0, -1.0, -2.0), (24, 24): (3.0, -1.0, -1.0), (24, 23): (3.0, -1.0, 0.0), (24, 22): (3.0, -1.0, 1.0), (24, 21): (3.0, -1.0, 2.0), (24, 20): (3.0, -1.0, 3.0), (23, 26): (3.0, 0.0, -3.0), (23, 25): (3.0, 0.0, -2.0), (23, 24): (3.0, 0.0, -1.0), (23, 23): (3.0, 0.0, 0.0), (23, 22): (3.0, 0.0, 1.0), (23, 21): (3.0, 0.0, 2.0), (23, 20): (3.0, 0.0, 3.0), (22, 26): (3.0, 1.0, -3.0), (22, 25): (3.0, 1.0, -2.0), (22, 24): (3.0, 1.0, -1.0), (22, 23): (3.0, 1.0, 0.0), (22, 22): (3.0, 1.0, 1.0), (22, 21): (3.0, 1.0, 2.0), (22, 20): (3.0, 1.0, 3.0), (21, 26): (3.0, 2.0, -3.0), (21, 25): (3.0, 2.0, -2.0), (21, 24): (3.0, 2.0, -1.0), (21, 23): (3.0, 2.0, 0.0), (21, 22): (3.0, 2.0, 1.0), (21, 21): (3.0, 2.0, 2.0), (21, 20): (3.0, 2.0, 3.0), (20, 26): (3.0, 3.0, -3.0), (20, 25): (3.0, 3.0, -2.0), (20, 24): (3.0, 3.0, -1.0), (20, 23): (3.0, 3.0, 0.0), (20, 22): (3.0, 3.0, 1.0), (20, 21): (3.0, 3.0, 2.0), (20, 20): (3.0, 3.0, 3.0), (19, 19): (4.0, -4.0, -4.0), (19, 18): (4.0, -4.0, -3.0), (19, 17): (4.0, -4.0, -2.0), (19, 16): (4.0, -4.0, -1.0), (19, 15): (4.0, -4.0, 0.0), (19, 14): (4.0, -4.0, 1.0), (19, 13): (4.0, -4.0, 2.0), (19, 12): (4.0, -4.0, 3.0), (19, 11): (4.0, -4.0, 4.0), (18, 19): (4.0, -3.0, -4.0), (18, 18): (4.0, -3.0, -3.0), (18, 17): (4.0, -3.0, -2.0), (18, 16): (4.0, -3.0, -1.0), (18, 15): (4.0, -3.0, 0.0), (18, 14): (4.0, -3.0, 1.0), (18, 13): (4.0, -3.0, 2.0), (18, 12): (4.0, -3.0, 3.0), (18, 11): (4.0, -3.0, 4.0), (17, 19): (4.0, -2.0, -4.0), (17, 18): (4.0, -2.0, -3.0), (17, 17): (4.0, -2.0, -2.0), (17, 16): (4.0, -2.0, -1.0), (17, 15): (4.0, -2.0, 0.0), (17, 14): (4.0, -2.0, 1.0), (17, 13): (4.0, -2.0, 2.0), (17, 12): (4.0, -2.0, 3.0), (17, 11): (4.0, -2.0, 4.0), (16, 19): (4.0, -1.0, -4.0), (16, 18): (4.0, -1.0, -3.0), (16, 17): (4.0, -1.0, -2.0), (16, 16): (4.0, -1.0, -1.0), (16, 15): (4.0, -1.0, 0.0), (16, 14): (4.0, -1.0, 1.0), (16, 13): (4.0, -1.0, 2.0), (16, 12): (4.0, -1.0, 3.0), (16, 11): (4.0, -1.0, 4.0), (15, 19): (4.0, 0.0, -4.0), (15, 18): (4.0, 0.0, -3.0), (15, 17): (4.0, 0.0, -2.0), (15, 16): (4.0, 0.0, -1.0), (15, 15): (4.0, 0.0, 0.0), (15, 14): (4.0, 0.0, 1.0), (15, 13): (4.0, 0.0, 2.0), (15, 12): (4.0, 0.0, 3.0), (15, 11): (4.0, 0.0, 4.0), (14, 19): (4.0, 1.0, -4.0), (14, 18): (4.0, 1.0, -3.0), (14, 17): (4.0, 1.0, -2.0), (14, 16): (4.0, 1.0, -1.0), (14, 15): (4.0, 1.0, 0.0), (14, 14): (4.0, 1.0, 1.0), (14, 13): (4.0, 1.0, 2.0), (14, 12): (4.0, 1.0, 3.0), (14, 11): (4.0, 1.0, 4.0), (13, 19): (4.0, 2.0, -4.0), (13, 18): (4.0, 2.0, -3.0), (13, 17): (4.0, 2.0, -2.0), (13, 16): (4.0, 2.0, -1.0), (13, 15): (4.0, 2.0, 0.0), (13, 14): (4.0, 2.0, 1.0), (13, 13): (4.0, 2.0, 2.0), (13, 12): (4.0, 2.0, 3.0), (13, 11): (4.0, 2.0, 4.0), (12, 19): (4.0, 3.0, -4.0), (12, 18): (4.0, 3.0, -3.0), (12, 17): (4.0, 3.0, -2.0), (12, 16): (4.0, 3.0, -1.0), (12, 15): (4.0, 3.0, 0.0), (12, 14): (4.0, 3.0, 1.0), (12, 13): (4.0, 3.0, 2.0), (12, 12): (4.0, 3.0, 3.0), (12, 11): (4.0, 3.0, 4.0), (11, 19): (4.0, 4.0, -4.0), (11, 18): (4.0, 4.0, -3.0), (11, 17): (4.0, 4.0, -2.0), (11, 16): (4.0, 4.0, -1.0), (11, 15): (4.0, 4.0, 0.0), (11, 14): (4.0, 4.0, 1.0), (11, 13): (4.0, 4.0, 2.0), (11, 12): (4.0, 4.0, 3.0), (11, 11): (4.0, 4.0, 4.0), (10, 10): (5.0, -5.0, -5.0), (10, 9): (5.0, -5.0, -4.0), (10, 8): (5.0, -5.0, -3.0), (10, 7): (5.0, -5.0, -2.0), (10, 6): (5.0, -5.0, -1.0), (10, 5): (5.0, -5.0, 0.0), (10, 4): (5.0, -5.0, 1.0), (10, 3): (5.0, -5.0, 2.0), (10, 2): (5.0, -5.0, 3.0), (10, 1): (5.0, -5.0, 4.0), (10, 0): (5.0, -5.0, 5.0), (9, 10): (5.0, -4.0, -5.0), (9, 9): (5.0, -4.0, -4.0), (9, 8): (5.0, -4.0, -3.0), (9, 7): (5.0, -4.0, -2.0), (9, 6): (5.0, -4.0, -1.0), (9, 5): (5.0, -4.0, 0.0), (9, 4): (5.0, -4.0, 1.0), (9, 3): (5.0, -4.0, 2.0), (9, 2): (5.0, -4.0, 3.0), (9, 1): (5.0, -4.0, 4.0), (9, 0): (5.0, -4.0, 5.0), (8, 10): (5.0, -3.0, -5.0), (8, 9): (5.0, -3.0, -4.0), (8, 8): (5.0, -3.0, -3.0), (8, 7): (5.0, -3.0, -2.0), (8, 6): (5.0, -3.0, -1.0), (8, 5): (5.0, -3.0, 0.0), (8, 4): (5.0, -3.0, 1.0), (8, 3): (5.0, -3.0, 2.0), (8, 2): (5.0, -3.0, 3.0), (8, 1): (5.0, -3.0, 4.0), (8, 0): (5.0, -3.0, 5.0), (7, 10): (5.0, -2.0, -5.0), (7, 9): (5.0, -2.0, -4.0), (7, 8): (5.0, -2.0, -3.0), (7, 7): (5.0, -2.0, -2.0), (7, 6): (5.0, -2.0, -1.0), (7, 5): (5.0, -2.0, 0.0), (7, 4): (5.0, -2.0, 1.0), (7, 3): (5.0, -2.0, 2.0), (7, 2): (5.0, -2.0, 3.0), (7, 1): (5.0, -2.0, 4.0), (7, 0): (5.0, -2.0, 5.0), (6, 10): (5.0, -1.0, -5.0), (6, 9): (5.0, -1.0, -4.0), (6, 8): (5.0, -1.0, -3.0), (6, 7): (5.0, -1.0, -2.0), (6, 6): (5.0, -1.0, -1.0), (6, 5): (5.0, -1.0, 0.0), (6, 4): (5.0, -1.0, 1.0), (6, 3): (5.0, -1.0, 2.0), (6, 2): (5.0, -1.0, 3.0), (6, 1): (5.0, -1.0, 4.0), (6, 0): (5.0, -1.0, 5.0), (5, 10): (5.0, 0.0, -5.0), (5, 9): (5.0, 0.0, -4.0), (5, 8): (5.0, 0.0, -3.0), (5, 7): (5.0, 0.0, -2.0), (5, 6): (5.0, 0.0, -1.0), (5, 5): (5.0, 0.0, 0.0), (5, 4): (5.0, 0.0, 1.0), (5, 3): (5.0, 0.0, 2.0), (5, 2): (5.0, 0.0, 3.0), (5, 1): (5.0, 0.0, 4.0), (5, 0): (5.0, 0.0, 5.0), (4, 10): (5.0, 1.0, -5.0), (4, 9): (5.0, 1.0, -4.0), (4, 8): (5.0, 1.0, -3.0), (4, 7): (5.0, 1.0, -2.0), (4, 6): (5.0, 1.0, -1.0), (4, 5): (5.0, 1.0, 0.0), (4, 4): (5.0, 1.0, 1.0), (4, 3): (5.0, 1.0, 2.0), (4, 2): (5.0, 1.0, 3.0), (4, 1): (5.0, 1.0, 4.0), (4, 0): (5.0, 1.0, 5.0), (3, 10): (5.0, 2.0, -5.0), (3, 9): (5.0, 2.0, -4.0), (3, 8): (5.0, 2.0, -3.0), (3, 7): (5.0, 2.0, -2.0), (3, 6): (5.0, 2.0, -1.0), (3, 5): (5.0, 2.0, 0.0), (3, 4): (5.0, 2.0, 1.0), (3, 3): (5.0, 2.0, 2.0), (3, 2): (5.0, 2.0, 3.0), (3, 1): (5.0, 2.0, 4.0), (3, 0): (5.0, 2.0, 5.0), (2, 10): (5.0, 3.0, -5.0), (2, 9): (5.0, 3.0, -4.0), (2, 8): (5.0, 3.0, -3.0), (2, 7): (5.0, 3.0, -2.0), (2, 6): (5.0, 3.0, -1.0), (2, 5): (5.0, 3.0, 0.0), (2, 4): (5.0, 3.0, 1.0), (2, 3): (5.0, 3.0, 2.0), (2, 2): (5.0, 3.0, 3.0), (2, 1): (5.0, 3.0, 4.0), (2, 0): (5.0, 3.0, 5.0), (1, 10): (5.0, 4.0, -5.0), (1, 9): (5.0, 4.0, -4.0), (1, 8): (5.0, 4.0, -3.0), (1, 7): (5.0, 4.0, -2.0), (1, 6): (5.0, 4.0, -1.0), (1, 5): (5.0, 4.0, 0.0), (1, 4): (5.0, 4.0, 1.0), (1, 3): (5.0, 4.0, 2.0), (1, 2): (5.0, 4.0, 3.0), (1, 1): (5.0, 4.0, 4.0), (1, 0): (5.0, 4.0, 5.0), (0, 10): (5.0, 5.0, -5.0), (0, 9): (5.0, 5.0, -4.0), (0, 8): (5.0, 5.0, -3.0), (0, 7): (5.0, 5.0, -2.0), (0, 6): (5.0, 5.0, -1.0), (0, 5): (5.0, 5.0, 0.0), (0, 4): (5.0, 5.0, 1.0), (0, 3): (5.0, 5.0, 2.0), (0, 2): (5.0, 5.0, 3.0), (0, 1): (5.0, 5.0, 4.0), (0, 0): (5.0, 5.0, 5.0)}
row = np.array([10])
col = np.array([10])
data = np.array([1])
state = csc_matrix((data, (row, col)), shape=(36, 36))
all_iks = {(0.0, -0.0, -0.0): (35, 35), (1.0, -1.0, -1.0): (34, 34), (1.0, -1.0, 0.0): (34, 33), (1.0, -1.0, 1.0): (34, 32), (1.0, 0.0, -1.0): (33, 34), (1.0, 0.0, 0.0): (33, 33), (1.0, 0.0, 1.0): (33, 32), (1.0, 1.0, -1.0): (32, 34), (1.0, 1.0, 0.0): (32, 33), (1.0, 1.0, 1.0): (32, 32), (2.0, -2.0, -2.0): (31, 31), (2.0, -2.0, -1.0): (31, 30), (2.0, -2.0, 0.0): (31, 29), (2.0, -2.0, 1.0): (31, 28), (2.0, -2.0, 2.0): (31, 27), (2.0, -1.0, -2.0): (30, 31), (2.0, -1.0, -1.0): (30, 30), (2.0, -1.0, 0.0): (30, 29), (2.0, -1.0, 1.0): (30, 28), (2.0, -1.0, 2.0): (30, 27), (2.0, 0.0, -2.0): (29, 31), (2.0, 0.0, -1.0): (29, 30), (2.0, 0.0, 0.0): (29, 29), (2.0, 0.0, 1.0): (29, 28), (2.0, 0.0, 2.0): (29, 27), (2.0, 1.0, -2.0): (28, 31), (2.0, 1.0, -1.0): (28, 30), (2.0, 1.0, 0.0): (28, 29), (2.0, 1.0, 1.0): (28, 28), (2.0, 1.0, 2.0): (28, 27), (2.0, 2.0, -2.0): (27, 31), (2.0, 2.0, -1.0): (27, 30), (2.0, 2.0, 0.0): (27, 29), (2.0, 2.0, 1.0): (27, 28), (2.0, 2.0, 2.0): (27, 27), (3.0, -3.0, -3.0): (26, 26), (3.0, -3.0, -2.0): (26, 25), (3.0, -3.0, -1.0): (26, 24), (3.0, -3.0, 0.0): (26, 23), (3.0, -3.0, 1.0): (26, 22), (3.0, -3.0, 2.0): (26, 21), (3.0, -3.0, 3.0): (26, 20), (3.0, -2.0, -3.0): (25, 26), (3.0, -2.0, -2.0): (25, 25), (3.0, -2.0, -1.0): (25, 24), (3.0, -2.0, 0.0): (25, 23), (3.0, -2.0, 1.0): (25, 22), (3.0, -2.0, 2.0): (25, 21), (3.0, -2.0, 3.0): (25, 20), (3.0, -1.0, -3.0): (24, 26), (3.0, -1.0, -2.0): (24, 25), (3.0, -1.0, -1.0): (24, 24), (3.0, -1.0, 0.0): (24, 23), (3.0, -1.0, 1.0): (24, 22), (3.0, -1.0, 2.0): (24, 21), (3.0, -1.0, 3.0): (24, 20), (3.0, 0.0, -3.0): (23, 26), (3.0, 0.0, -2.0): (23, 25), (3.0, 0.0, -1.0): (23, 24), (3.0, 0.0, 0.0): (23, 23), (3.0, 0.0, 1.0): (23, 22), (3.0, 0.0, 2.0): (23, 21), (3.0, 0.0, 3.0): (23, 20), (3.0, 1.0, -3.0): (22, 26), (3.0, 1.0, -2.0): (22, 25), (3.0, 1.0, -1.0): (22, 24), (3.0, 1.0, 0.0): (22, 23), (3.0, 1.0, 1.0): (22, 22), (3.0, 1.0, 2.0): (22, 21), (3.0, 1.0, 3.0): (22, 20), (3.0, 2.0, -3.0): (21, 26), (3.0, 2.0, -2.0): (21, 25), (3.0, 2.0, -1.0): (21, 24), (3.0, 2.0, 0.0): (21, 23), (3.0, 2.0, 1.0): (21, 22), (3.0, 2.0, 2.0): (21, 21), (3.0, 2.0, 3.0): (21, 20), (3.0, 3.0, -3.0): (20, 26), (3.0, 3.0, -2.0): (20, 25), (3.0, 3.0, -1.0): (20, 24), (3.0, 3.0, 0.0): (20, 23), (3.0, 3.0, 1.0): (20, 22), (3.0, 3.0, 2.0): (20, 21), (3.0, 3.0, 3.0): (20, 20), (4.0, -4.0, -4.0): (19, 19), (4.0, -4.0, -3.0): (19, 18), (4.0, -4.0, -2.0): (19, 17), (4.0, -4.0, -1.0): (19, 16), (4.0, -4.0, 0.0): (19, 15), (4.0, -4.0, 1.0): (19, 14), (4.0, -4.0, 2.0): (19, 13), (4.0, -4.0, 3.0): (19, 12), (4.0, -4.0, 4.0): (19, 11), (4.0, -3.0, -4.0): (18, 19), (4.0, -3.0, -3.0): (18, 18), (4.0, -3.0, -2.0): (18, 17), (4.0, -3.0, -1.0): (18, 16), (4.0, -3.0, 0.0): (18, 15), (4.0, -3.0, 1.0): (18, 14), (4.0, -3.0, 2.0): (18, 13), (4.0, -3.0, 3.0): (18, 12), (4.0, -3.0, 4.0): (18, 11), (4.0, -2.0, -4.0): (17, 19), (4.0, -2.0, -3.0): (17, 18), (4.0, -2.0, -2.0): (17, 17), (4.0, -2.0, -1.0): (17, 16), (4.0, -2.0, 0.0): (17, 15), (4.0, -2.0, 1.0): (17, 14), (4.0, -2.0, 2.0): (17, 13), (4.0, -2.0, 3.0): (17, 12), (4.0, -2.0, 4.0): (17, 11), (4.0, -1.0, -4.0): (16, 19), (4.0, -1.0, -3.0): (16, 18), (4.0, -1.0, -2.0): (16, 17), (4.0, -1.0, -1.0): (16, 16), (4.0, -1.0, 0.0): (16, 15), (4.0, -1.0, 1.0): (16, 14), (4.0, -1.0, 2.0): (16, 13), (4.0, -1.0, 3.0): (16, 12), (4.0, -1.0, 4.0): (16, 11), (4.0, 0.0, -4.0): (15, 19), (4.0, 0.0, -3.0): (15, 18), (4.0, 0.0, -2.0): (15, 17), (4.0, 0.0, -1.0): (15, 16), (4.0, 0.0, 0.0): (15, 15), (4.0, 0.0, 1.0): (15, 14), (4.0, 0.0, 2.0): (15, 13), (4.0, 0.0, 3.0): (15, 12), (4.0, 0.0, 4.0): (15, 11), (4.0, 1.0, -4.0): (14, 19), (4.0, 1.0, -3.0): (14, 18), (4.0, 1.0, -2.0): (14, 17), (4.0, 1.0, -1.0): (14, 16), (4.0, 1.0, 0.0): (14, 15), (4.0, 1.0, 1.0): (14, 14), (4.0, 1.0, 2.0): (14, 13), (4.0, 1.0, 3.0): (14, 12), (4.0, 1.0, 4.0): (14, 11), (4.0, 2.0, -4.0): (13, 19), (4.0, 2.0, -3.0): (13, 18), (4.0, 2.0, -2.0): (13, 17), (4.0, 2.0, -1.0): (13, 16), (4.0, 2.0, 0.0): (13, 15), (4.0, 2.0, 1.0): (13, 14), (4.0, 2.0, 2.0): (13, 13), (4.0, 2.0, 3.0): (13, 12), (4.0, 2.0, 4.0): (13, 11), (4.0, 3.0, -4.0): (12, 19), (4.0, 3.0, -3.0): (12, 18), (4.0, 3.0, -2.0): (12, 17), (4.0, 3.0, -1.0): (12, 16), (4.0, 3.0, 0.0): (12, 15), (4.0, 3.0, 1.0): (12, 14), (4.0, 3.0, 2.0): (12, 13), (4.0, 3.0, 3.0): (12, 12), (4.0, 3.0, 4.0): (12, 11), (4.0, 4.0, -4.0): (11, 19), (4.0, 4.0, -3.0): (11, 18), (4.0, 4.0, -2.0): (11, 17), (4.0, 4.0, -1.0): (11, 16), (4.0, 4.0, 0.0): (11, 15), (4.0, 4.0, 1.0): (11, 14), (4.0, 4.0, 2.0): (11, 13), (4.0, 4.0, 3.0): (11, 12), (4.0, 4.0, 4.0): (11, 11), (5.0, -5.0, -5.0): (10, 10), (5.0, -5.0, -4.0): (10, 9), (5.0, -5.0, -3.0): (10, 8), (5.0, -5.0, -2.0): (10, 7), (5.0, -5.0, -1.0): (10, 6), (5.0, -5.0, 0.0): (10, 5), (5.0, -5.0, 1.0): (10, 4), (5.0, -5.0, 2.0): (10, 3), (5.0, -5.0, 3.0): (10, 2), (5.0, -5.0, 4.0): (10, 1), (5.0, -5.0, 5.0): (10, 0), (5.0, -4.0, -5.0): (9, 10), (5.0, -4.0, -4.0): (9, 9), (5.0, -4.0, -3.0): (9, 8), (5.0, -4.0, -2.0): (9, 7), (5.0, -4.0, -1.0): (9, 6), (5.0, -4.0, 0.0): (9, 5), (5.0, -4.0, 1.0): (9, 4), (5.0, -4.0, 2.0): (9, 3), (5.0, -4.0, 3.0): (9, 2), (5.0, -4.0, 4.0): (9, 1), (5.0, -4.0, 5.0): (9, 0), (5.0, -3.0, -5.0): (8, 10), (5.0, -3.0, -4.0): (8, 9), (5.0, -3.0, -3.0): (8, 8), (5.0, -3.0, -2.0): (8, 7), (5.0, -3.0, -1.0): (8, 6), (5.0, -3.0, 0.0): (8, 5), (5.0, -3.0, 1.0): (8, 4), (5.0, -3.0, 2.0): (8, 3), (5.0, -3.0, 3.0): (8, 2), (5.0, -3.0, 4.0): (8, 1), (5.0, -3.0, 5.0): (8, 0), (5.0, -2.0, -5.0): (7, 10), (5.0, -2.0, -4.0): (7, 9), (5.0, -2.0, -3.0): (7, 8), (5.0, -2.0, -2.0): (7, 7), (5.0, -2.0, -1.0): (7, 6), (5.0, -2.0, 0.0): (7, 5), (5.0, -2.0, 1.0): (7, 4), (5.0, -2.0, 2.0): (7, 3), (5.0, -2.0, 3.0): (7, 2), (5.0, -2.0, 4.0): (7, 1), (5.0, -2.0, 5.0): (7, 0), (5.0, -1.0, -5.0): (6, 10), (5.0, -1.0, -4.0): (6, 9), (5.0, -1.0, -3.0): (6, 8), (5.0, -1.0, -2.0): (6, 7), (5.0, -1.0, -1.0): (6, 6), (5.0, -1.0, 0.0): (6, 5), (5.0, -1.0, 1.0): (6, 4), (5.0, -1.0, 2.0): (6, 3), (5.0, -1.0, 3.0): (6, 2), (5.0, -1.0, 4.0): (6, 1), (5.0, -1.0, 5.0): (6, 0), (5.0, 0.0, -5.0): (5, 10), (5.0, 0.0, -4.0): (5, 9), (5.0, 0.0, -3.0): (5, 8), (5.0, 0.0, -2.0): (5, 7), (5.0, 0.0, -1.0): (5, 6), (5.0, 0.0, 0.0): (5, 5), (5.0, 0.0, 1.0): (5, 4), (5.0, 0.0, 2.0): (5, 3), (5.0, 0.0, 3.0): (5, 2), (5.0, 0.0, 4.0): (5, 1), (5.0, 0.0, 5.0): (5, 0), (5.0, 1.0, -5.0): (4, 10), (5.0, 1.0, -4.0): (4, 9), (5.0, 1.0, -3.0): (4, 8), (5.0, 1.0, -2.0): (4, 7), (5.0, 1.0, -1.0): (4, 6), (5.0, 1.0, 0.0): (4, 5), (5.0, 1.0, 1.0): (4, 4), (5.0, 1.0, 2.0): (4, 3), (5.0, 1.0, 3.0): (4, 2), (5.0, 1.0, 4.0): (4, 1), (5.0, 1.0, 5.0): (4, 0), (5.0, 2.0, -5.0): (3, 10), (5.0, 2.0, -4.0): (3, 9), (5.0, 2.0, -3.0): (3, 8), (5.0, 2.0, -2.0): (3, 7), (5.0, 2.0, -1.0): (3, 6), (5.0, 2.0, 0.0): (3, 5), (5.0, 2.0, 1.0): (3, 4), (5.0, 2.0, 2.0): (3, 3), (5.0, 2.0, 3.0): (3, 2), (5.0, 2.0, 4.0): (3, 1), (5.0, 2.0, 5.0): (3, 0), (5.0, 3.0, -5.0): (2, 10), (5.0, 3.0, -4.0): (2, 9), (5.0, 3.0, -3.0): (2, 8), (5.0, 3.0, -2.0): (2, 7), (5.0, 3.0, -1.0): (2, 6), (5.0, 3.0, 0.0): (2, 5), (5.0, 3.0, 1.0): (2, 4), (5.0, 3.0, 2.0): (2, 3), (5.0, 3.0, 3.0): (2, 2), (5.0, 3.0, 4.0): (2, 1), (5.0, 3.0, 5.0): (2, 0), (5.0, 4.0, -5.0): (1, 10), (5.0, 4.0, -4.0): (1, 9), (5.0, 4.0, -3.0): (1, 8), (5.0, 4.0, -2.0): (1, 7), (5.0, 4.0, -1.0): (1, 6), (5.0, 4.0, 0.0): (1, 5), (5.0, 4.0, 1.0): (1, 4), (5.0, 4.0, 2.0): (1, 3), (5.0, 4.0, 3.0): (1, 2), (5.0, 4.0, 4.0): (1, 1), (5.0, 4.0, 5.0): (1, 0), (5.0, 5.0, -5.0): (0, 10), (5.0, 5.0, -4.0): (0, 9), (5.0, 5.0, -3.0): (0, 8), (5.0, 5.0, -2.0): (0, 7), (5.0, 5.0, -1.0): (0, 6), (5.0, 5.0, 0.0): (0, 5), (5.0, 5.0, 1.0): (0, 4), (5.0, 5.0, 2.0): (0, 3), (5.0, 5.0, 3.0): (0, 2), (5.0, 5.0, 4.0): (0, 1), (5.0, 5.0, 5.0): (0, 0)}
j_min = 0
j_max = 5
N_in = 10
d_dicke = 36

In [95]:

profile = line_profiler.LineProfiler(calc_rho_0)
profile.runcall(calc_rho_0,iks,jmm1,state,all_iks,j_min,j_max,N_in,d_dicke)
profile.print_stats()

<36x36 sparse matrix of type '<class 'numpy.complex128'>'
	with 2 stored elements in Compressed Sparse Column format>

Timer unit: 1e-06 s

Total time: 0.00584 s
File: /home/viet/.cache/ipython/cython/_cython_magic_853f609e8af478c6ba388641422f934d.pyx
Function: calc_rho_0 at line 63

Line #      Hits         Time  Per Hit   % Time  Line Contents
    63                                           def calc_rho_0(iks,jmm1,state,all_iks,j_min,j_max,N_in,d_dicke):
    64                                               # cdef np.ndarray[np.complex128_t, ndim=2] first_term
    65                                               # cdef np.ndarray[np.complex128_t, ndim=2] second_term
    66                                               # cdef np.ndarray[np.complex128_t, ndim=2] third_term
    67                                               # cdef np.ndarray[np.complex128_t, ndim=2] gamma_1
    68                                               # cdef np.ndarray[np.complex128_t, ndim=2] gamma_2
    69                                               # cdef np.ndarray[np.complex128_t, ndim=2] gamma_3
    70                 